In [7]:
%use kandy
%use dataframe

Define basic properties for simulation.
'startDate' and 'endDate' could be changed because of all assets' availability.

In [8]:
import portfolio.rebalancer.strategy.SPYOnlyStrategy
import java.time.ZoneId
import java.time.ZonedDateTime

val strategy = SPYOnlyStrategy()
val initialBudget = 100000.0
val startDate = ZonedDateTime.of(1970, 1, 1, 0, 0, 0, 0, ZoneId.systemDefault())
// val startDate = ZonedDateTime.of(2011, 4, 1, 0, 0, 0, 0, ZoneId.systemDefault())
val endDate = ZonedDateTime.of(2024, 1, 1, 0, 0, 0, 0, ZoneId.systemDefault())

Load all the CSV files including price data of various ETFs

In [9]:
import portfolio.rebalancer.io.PriceDataManager

val csvFilesDir = System.getProperty("user.dir") + "../csv"
val allPrices = PriceDataManager(baseDir = csvFilesDir).allPrices

Simulate HAA strategy in as longest period where all the assets are available as possible

In [10]:
import portfolio.rebalancer.StrategySimulator
import portfolio.rebalancer.dto.SymbolPricesByDate

val strategySimulator = StrategySimulator()
val result = strategySimulator.simulate(
    budget = initialBudget,
    strategy = strategy,
    allTimeSymbolPrices = SymbolPricesByDate(value = allPrices),
    startDate = startDate,
    endDate = endDate,
)

Start simulating from 1993-01-30T06:00+09:00[Asia/Seoul] with 100000.0 USD
datesToReBalance=[1994-01-29T06:00+09:00[Asia/Seoul]]


Total profit rate

In [11]:
"${(result.timeSeriesData.last().second / initialBudget - 1.0) * 100}% between ${result.timeSeriesData.first().first} and ${result.timeSeriesData.last().first}"

1613.018288172% between 1993-01-30T06:00+09:00[Asia/Seoul] and 2023-12-30T06:00+09:00[Asia/Seoul]

Plot a line graph showing total asset over time

In [12]:
import java.time.format.DateTimeFormatter

val formatter = DateTimeFormatter.ofPattern("yyyy-MM-dd")

val dates = result.timeSeriesData.map { it.first.format(formatter) }
val assets = result.timeSeriesData.map { it.second }

mapOf(
    "date" to dates,
    "asset" to assets,
).plot {
    layout { 
        title = "Total asset over time"
        size = 1200 to 400
     }
    line {
        x(dates)
        y(assets)
        color = Color.PURPLE
        width = 1.0
    }
}

Drawdowns over time

In [15]:
import portfolio.rebalancer.util.DrawdownCalculator

val dates = result.timeSeriesData.map { it.first.format(formatter) }.drop(1)
val drawdowns = DrawdownCalculator().calculateDrawdowns(result.timeSeriesData.map { it.second })
val mdds = drawdowns.scan(0.0) { acc, x -> minOf(acc, x) }.drop(1)

mapOf(
    "date" to dates + dates,
    "drawdown" to drawdowns + mdds,
    "line" to List(dates.size) { "Drawdown" } + List(dates.size) { "MDD" },
).plot {
    groupBy("line") {
        line {
            x("date")
            y("drawdown")
            color("line")
        }
    }
    layout {
        title = "Drawdown and MDD over time"
        size = 1200 to 400
    }
}